<h1>Chapter 2 - Tokens and Token Embeddings</h1>
<i>Exploring tokens and embeddings as an integral part of building LLMs</i>


<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter02/Chapter%202%20-%20Tokens%20and%20Token%20Embeddings.ipynb)

---

This notebook is for Chapter 2 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [4]:
# %%capture
#!pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

zsh:1: 4.41.2 not found


## 第一部分：理解语言模型
在本书的第一部分中，我们深入探讨了小型和大型语言模型的内部工作机制。本部分将介绍许多贯穿全书使用的术语和定义。  
我们首先概述了该领域和常用技术（见第1章）  
然后转向这些模型的两个核心组成部分：分词和嵌入（见第2章）。  
本书这一部分以Jay著名的《图解Transformer》的更新和扩展版作为结尾，该版本深入探讨了这些模型的架构（见第3章）。
## 第二部分：使用预训练语言模型
在本书的第二部分中，我们将通过常见的用例来探讨如何使用大型语言模型（LLMs）。我们利用预训练模型，并展示其无需微调即可实现的功能。学习这些单独的语言模型功能将使你掌握使用大型语言模型解决问题的技能集，并能够构建越来越高级的系统和流程。   
你将学习如何使用语言模型进行监督分类（见第4章）  
文本聚类和主题建模（见第5章）  
利用嵌入模型进行语义搜索（见第6章）  
生成文本（见第7章和第8章）  
以及将文本生成的能力扩展到视觉领域（见第9章） 
## 第三部分：训练与微调语言模型
在本书的第三部分中，我们通过训练与微调各类语言模型来探索高级概念。  
我们将探讨如何创建和微调嵌入模型（见第10章）  
回顾如何为分类任务微调BERT（见第11章） 
并以几种微调生成模型的方法结束本书（见第12章）。  

# Tokens and Embeddings


![](1.png)

大语言模型LLM读取文字的过程
>1. tokens  
2. tokeniazation methods  
3. embeddings  
4. word2vec embedding method
5. extend the concept of token embeddings--recommendation systems 

# LLM Tokenization

![](2.png)


>token 文本的离散化单位 （令牌）（词元）（标记）

![](3.png)

>https://platform.openai.com/tokenizer

>[阿里云token计算器](https://dashscope.console.aliyun.com/tokenizer)

# Downloading and Running An LLM

The first step is to load our model onto the GPU for faster inference. Note that we load the model and tokenizer separately and keep them as such so that we can explore them separately.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

<s> Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: My Sincere Apologies for the Gardening Mishap

Dear


In [ ]:
print(input_ids)

tensor([[    1, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


>tensor 张量 向量和矩阵的集合    

>[ 50, 100, 150 ]   
[ 75, 120, 200 ]  
[ 90, 110, 180 ] 



>[  
  [[ 50,  25, 100], [100,  75, 150], [150, 125, 200]],  
  [[ 75,  50, 125], [120,  95, 170], [200, 175, 240]],  
  [[100,  75, 150], [150, 125, 200], [180, 155, 220]]
]  

![](4.png)

In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

<s>
Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


- The first token is ID 1 (<s\>), a special token indicating the beginning of the text.
- Some tokens are complete words (e.g., Write, an, email).
- Some tokens are parts of words (e.g., apolog, izing, trag, ic).
- Punctuation characters are their own token.  
“注意空格字符没有自己的标记。相反，部分标记(如“izing”和“ic”)在其开头有一个特殊的隐藏字符，表明它们与文本中前面的标记相连接。没有这个特殊字符的标记被认为在它们前面有一个空格。”

In [ ]:
generation_output

tensor([[    1, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,  1619,   317,
          3742,   406,  6225, 11763,   363,   278, 19906,   292,   341,   728,
           481,    13,    13, 29928,   799]], device='cuda:0')

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# How Does the Tokenizer Break Down Text

分词器如何分解输入提示，主要受三个主要因素的影响。  
首先，在模型设计阶段，模型的创建者会选择一种**分词方法**。流行的方法包括Byte-Pair Encoding(BPE）（GPT模型广泛使用）和WordPiece（BERT使用）。这些方法在目标上是相似的，即旨在优化一组高效的词元来代表一个文本数据集，但它们实现这一目标的方式不同。  
其次，在选择方法之后，我们需要做出一系列**分词器设计选择**，比如词汇表的大小以及使用哪些特殊词元。关于这些内容的更多信息，请参见‘比较训练后的LLM分词器’。  
第三，分词器需要在特定的数据集上进行**训练**，以建立能够最佳表示该数据集的词汇表。即使我们设置了相同的‘方法和参数’，一个在英文文本数据集上训练的分词器也会与在代码数据集或多语言文本数据集上训练的分词器不同。  

子词分词器BPE和WordPiece理解 https://blog.csdn.net/beingstrong/article/details/130169542

除了用于输入文本处理成语言模型的输入之外，分词器还被用于语言模型的输出，将生成的词元ID转换为与之关联的输出单词或词元，如下图所示。

![](13.png)

# “Word vs . Subword vs . Character vs . Byte token”

>单词词元（word token）：这种方法在早期的技术中很常见，比如word2vec，但在自然语言处理（NLP）中的使用越来越少。然而，由于其有用性，它仍被用于NLP之外的场景，如推荐系统，我们将在本章后面看到。
单词分词的一个挑战是，分词器可能无法处理在分词器训练后新进入数据集的单词。这也导致词汇表中存在许多差异极小的词元（例如，apology、apologize、apologetic、apologist）。后一个挑战通过子词分词得到解决，因为子词分词有一个表示“apolog”的词元，然后还有常见的后缀词元（例如，-y、-ize、-etic、-ist），这些后缀词元与许多其他词元共用，从而形成了一个更具表达力的词汇表。”

>子词词元(subword token)
这种方法包含完整单词和部分单词。除了之前提到的词汇表达力外，该方法的另一个好处是能够通过将新词元分解为更小的字符（这些字符往往是词汇表的一部分）来表示新词。

>字母词元（character tokens）
“这是另一种能够成功处理新词的方法，因为它可以依赖原始字母。虽然这使得表示更容易进行分词，但却增加了建模的难度。使用子词分词的模型可以将‘play’表示为一个词元，而使用字符级词元的模型则需要除了对序列的其余部分进行建模外，还需要对拼写‘p-l-a-y’的信息进行建模。
在能够在Transformer模型的有限上下文长度内容纳更多文本方面，子词词元相较于字符词元具有优势。因此，在上下文长度为1024的模型中，使用子词分词可以容纳的文本量大约是使用字符词元的三倍（子词词元的每个词元平均通常有三个字符）。”

>字节词元（Byte tokens）
“另一种分词方法是将词元分解为用于表示Unicode字符的单个字节。像《CANINE：为语言表示预训练一个高效的无需分词的编码器》这样的论文概述了这样的方法，也被称为“无需分词的编码”。其他工作，如《ByT5：使用预训练的字节到字节模型迈向无需词元的未来》，表明这可以成为一种具有竞争力的方法，特别是在多语言场景中。

![](5.png)

# Comparing Trained LLM Tokenizers


“我们在前面已经指出了决定标记器中出现的标记的三个主要因素:标记方法，我们用于初始化标记器的参数和特殊标记，以及标记器所训练的数据集。让我们比较和对比一些实际的、训练过的标记器，看看这些选择是如何改变它们的行为的。这种比较将向我们展示，较新的标记器已经改变了它们的行为以提高模型性能，我们还将看到专门的模型(例如代码生成模型)通常需要专门的标记器。”


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

>对于每个标记，使用 ANSI 转义序列设置背景色。转义序列的格式为 \x1b[0;30;48;2;{color_value}m，其中：
\x1b 是转义字符（ESC）。
[0;30;48;2;{color_value}] 是控制序列，用于设置文本属性：
0 重置/忽略所有先前的属性。  
30 设置前景色。  
48 设置背景色。  
2 表示使用24位RGB颜色。  
{color_value} 是从 colors_list 中选取的颜色值，通过 idx % len(colors_list) 实现循环选取。

In [17]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

这将使我们能够观察每种标记器如何处理多种不同类型的标记：
1. 大写字母。
2. 非英语语言。
3. 表情符号。
4. 带有关键字和常用于缩进（例如在Python等语言中）的空格的编程代码。
5. 数字和数位。
6. 特殊标记。这些是除了表示文本之外还有其他作用的独特标记。它们包括指示文本开始或结束的标记（这是模型向系统发出已完成此生成的信号的方式），以及我们将看到的其他功能。
让我们从较旧的标记器到较新的标记器进行探索，看看它们如何对这段文本进行标记化，以及这可能揭示出关于语言模型的什么信息。我们将对文本进行标记化，然后使用这个函数为每个标记打印带有背景颜色的文本：

BERT  uncased （全部转换为小写)  cased（保留大小写） 

In [ ]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " / t / t " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

>词汇表大小（Vocabulary size）: 30,522  
这意味着标记器能够识别和编码的唯一标记（或称为“单词”、“子词”或“标记单元”）的数量是30,522个。  
1.未知标记（unk_token [UNK]）:  
当标记器遇到一个它不认识的标记时，它会使用这个特殊标记来代替。这通常发生在处理训练数据中未出现的单词时。  
2.分隔标记（sep_token [SEP]）:  
这个标记用于分隔输入中的不同部分，特别是在需要将两个文本片段作为单个输入传递给模型时（例如，在问答或句子对分类任务中）。它允许模型理解输入的两个部分应该被分开处理。  
3.填充标记（pad_token [PAD]）: 
在处理不同长度的输入时，这个标记用于填充较短的输入，使它们达到模型期望的固定长度。这样，模型就可以以批处理的方式高效地处理多个输入。  
4.分类标记（cls_token [CLS]）:  
在执行分类任务时，这个特殊标记通常被添加到输入的开头。模型的最终输出（特别是对应于这个标记的隐藏状态）被用作分类任务的输入。  
5.掩码标记（mask_token [MASK]）:  
在训练过程中，特别是在使用掩码语言模型（Masked Language Model, MLM）时，这个标记用于替换输入中的某些标记，然后模型尝试预测这些被替换的标记是什么。这是预训练模型如BERT中的一个关键步骤。  

> 1. 换行符的消失：
在uncased版本的BERT分词器中，换行符被移除。这意味着模型无法获取到原本通过换行符编码的信息。例如，在聊天记录中，如果每一轮对话都位于新的一行，那么模型就无法区分这些对话的轮次。
2. 全部文本转换为小写：
如其名，uncased版本的BERT将所有文本转换为小写。这有助于减少词汇表的规模，因为同一个单词的不同大小写形式都被视为同一个单词。然而，这也可能导致一些语义信息的丢失，因为大小写有时在语言中承载着重要的含义（如专有名词、标题等）。
3. 子词编码：
在BERT中，较长的单词可能会被拆分成更小的子词单元。您提到的“capitalization”被编码为“##ization”和“capital”两个子词是一个典型的例子。这里的“##”前缀表示该子词是前一个子词的延续部分。这种方法也用于指示空格的位置，因为假设没有“##”前缀的子词前面有一个空格。
4. 表情符号和中文字符的替换：
在BERT的原始预训练版本中，由于词汇表大小的限制和训练数据的特性，一些特殊字符（如表情符号）和某些语言的字符（如中文）可能没有被包括在内。因此，这些字符在输入模型时会被替换为[UNK]特殊标记，表示“未知标记”。这意味着模型无法直接理解和处理这些字符。

In [ ]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [21]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

>CA ##PI##TA##L##I##Z##AT##ION。  
BERT的分词器会将输入文本包裹在一个起始令牌和一个结束令牌[SEP]之间，同时还会添加一个[CLS]令牌。[CLS]和[SEP]是实用令牌，用于包裹输入文本，并各自有其特定的用途。[CLS]代表分类，因为它是用于句子分类的令牌。而[SEP]代表分隔符，它在需要将两个句子传递给模型的应用中被用来分隔句子（例如，在第8章中，我们将使用[SEP]令牌来分隔查询文本和候选结果文本）。

In [ ]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [22]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

词汇量:50257   
使用GPT-2标记器，我们注意到以下内容:  
换行符在分词器中表示。  
保留大写，单词“资本化”用四个记号表示。  
🎵鸟字符现在由多个token分别表示。虽然我们看到这些令牌打印为 � 字符，它们实际上代表不同的符号。例如，🎵表情符号被分解成令牌id为8582、236和113的令牌。标记器成功地从这些标记重构了原始字符。我们可以通过打印看到这一点 Tokenizer.decode ([8582, 236, 113])，打印出🎵。
两个制表符表示为两个标记(该词汇表中的标记号为197)，四个空格表示为三个标记(编号为220)，最后一个空格是结束引号字符的标记的一部分。

In [ ]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [23]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

“没有换行符或空白符号;这会给模型处理代码带来挑战。
表情符号和汉字都被 < unk > Token，让模型完全看不见它们。”

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [24]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

“GPT-4标记器的行为与其祖先GPT-2标记器相似。一些不同之处是:    
GPT-4标记器将四个空格表示为单个标记。事实上，它对每个空格序列都有一个特定的令牌，最多有83个空格。    
Python keyword  elif 在GPT-4中有自己的令牌。这一点和前一点都源于该模型除了自然语言之外对代码的关注。   
GPT-4标记器使用较少的标记来表示大多数单词。这里的例子包括“大写”(两个token对四个)和“token”(一个token对三个)。   
请参考我们所说的关于Ł令牌的GPT-2令牌器。

In [ ]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

“词汇量:49152  
特殊令牌示例:  
< | endoftext | >  
填写中间的令牌:  
< fim_prefix >  
< fim_middle >  
< fim_suffix >  
< fim_pad >  
在表示代码时，管理上下文很重要。一个文件可能对另一个文件中定义的函数进行函数调用。因此，模型需要某种方法能够识别同一代码库中不同文件中的代码，同时区分不同代码库中的代码。这就是为什么StarCoder2使用特殊的令牌来表示存储库的名称和文件名:  
<文件名>  
< reponame >  
< gh_stars >  

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s>  
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

前面关于已训练分词器的引导之旅展示了实际分词器之间存在的多种差异。但是什么决定了它们的分词行为呢？有三个主要的设计选择决定了分词器如何分解文本：分词方法、初始化参数以及分词器所针对的数据领域。  
>分词方法  
我们已经看到，存在多种分词方法，其中Byte-Pair Encoding（BPE）(逐步合并出现频率最高的字词生成词汇表)更为流行。这些方法中的每一种都概述了一个算法，用于如何选择一组适当的标记来表示数据集。你可以在Hugging Face页面上找到所有这些方法的精彩概述，该页面总结了分词器。

>分词器参数   
在选择分词方法之后，大型语言模型（LLM）设计者需要就分词器的参数做出一些决策。这些参数包括：  
词汇量大小：分词器的词汇表中要保留多少个tokens？（30K和50K常被用作词汇量值，但我们现在越来越多地看到更大的值，如100K。）  
特殊标记：我们希望模型跟踪哪些特殊标记？我们可以根据需要添加任意数量的这些标记，特别是如果我们想为特殊用例构建LLM时。常见选择包括：  
文本开始标记（例如，< s >）   
文本结束标记  
填充标记  
未知标记   
CLS标记   
掩码标记  
除了这些之外，LLM设计者还可以添加有助于更好地模拟他们试图关注的问题领域的标记，就像我们在Galactica的&lt;work&gt;和[START_REF]标记中所看到的那样。  
大写字母处理：在像英语这样的语言中，我们该如何处理大写字母？我们应该把所有内容都转换为小写吗？（名称大写通常携带有用信息，但我们是否希望浪费标记词汇空间来存储单词的全大写版本？）  

>数据集 
即使我们选择相同的方法和参数，分词器的行为也会因其训练所用的数据集而不同（甚至在我们开始模型训练之前）。前面提到的分词方法通过优化词汇表来表示特定数据集来工作。从我们的引导之旅中，我们已经看到这对像代码和多语言文本这样的数据集有何影响。  

![](14.png)

# token embedding
>embeddings是一种将token映射到向量空间中的点的方法，以捕捉它们之间的语义和语法关系。这些向量通常是通过训练模型（如Word2Vec、GloVe、BERT等）学习得到的，并用于各种NLP任务，如文本分类、命名实体识别、情感分析等。

“接下来，我们要解决的是如何为这些标记找到最佳的数值表示，这样模型就可以利用它们来计算并正确地模拟文本中的模式。这些模式向我们展示了模型在特定语言中的连贯性、编码能力，或是我们对语言模型所期望的日益增多的能力中的任何一种。
正如我们在第1章中看到的，嵌入（embeddings）正是为了这个目的而存在的。它们是用于捕捉语言中意义和模式的数值表示空间。

1.如果训练数据包含大量英文文本，那么该模式就会显现为一种能够表示和生成英语语言的模型。  
2.如果训练数据包含事实信息（例如，维基百科），那么该模型将具备生成一些事实信息的能力（参见以下注释）。

>oops：虽然模型在语言连贯性和事实生成的准确性上达到了不错的水平，但这却带来了一个新问题。一些用户开始信任模型的事实生成能力（例如，在2023年初，一些语言模型被冠以“谷歌杀手”的称号）。但很快，高级用户就意识到，仅依靠生成模型并不可靠作为搜索引擎。这促使了检索增强生成（RAG，Retrieval-Augmented Generation）的兴起，它将搜索和大型语言模型（LLMs）结合起来。我们将在第8章中更详细地介绍RAG

![](6.png)

# Contextualized Word Embeddings From a Language Model (Like BERT)

既然我们已经介绍了将标记嵌入作为语言模型的输入，接下来让我们看看语言模型如何创建更好的标记嵌入。这是使用语言模型进行文本表示的主要方式之一。这为命名实体识别或抽取式文本摘要（通过突出显示文本中最重要的部分来总结长文本，而不是生成新的文本作为摘要）等应用提供了强大支持。

语言模型不是用静态向量来表示每个标记或单词，而是创建上下文化词嵌入（如图2-8所示），这些嵌入根据上下文用不同的标记来表示同一个单词。然后，其他系统可以使用这些向量来完成各种任务。除了我们在上一段中提到的文本应用外，这些上下文化向量还为DALL·E、Midjourney和Stable Diffusion等AI图像生成系统提供了动力。

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

这段代码下载了一个预训练的分词器和模型，然后使用它们来处理字符串“Hello world”。模型的输出被保存在output变量中。让我们首先通过打印其维度（我们期望它是一个多维数组）来检查这个变量：

In [ ]:
output.shape

torch.Size([1, 4, 384])

跳过第一维，我们可以理解为有四个标记，每个标记都嵌入在一个384个值的向量中。第一维是批处理维度，用于我们想要同时向模型发送多个输入句子的情况（如训练时）（它们会同时处理，从而加快处理速度）。

但这四个向量是什么呢？是分词器将这两个单词拆分成了四个标记，还是这里发生了其他情况？我们可以利用所学到的关于分词器的知识来检查它们：

In [ ]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]


这个特定的分词器和模型通过在字符串的开始和结束处添加[CLS]和[SEP]标记来工作。

我们的语言模型现在已经处理了文本输入。其输出结果如下：

In [ ]:
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

这是语言模型的原始输出。大型语言模型的应用都是基于这样的输出构建的。

在图2-9中，我们概括了输入的分词处理以及语言模型产生的输出结果。从技术上讲，从标记ID转换为原始嵌入是语言模型内部发生的第一步。

![](7.png)

# Text Embeddings (For Sentences and Whole Documents)

# 文本嵌入向量

生成文本嵌入向量的方法有多种。其中最常见的方法之一是计算模型生成的所有标记嵌入的值的平均值。然而，高质量的文本嵌入模型往往是专门为文本嵌入任务训练的。

我们可以使用sentence-transformers包来生成文本嵌入，这是一个流行的用于利用预训练嵌入模型的包。与上一章中的transformers包一样，该包可以用于加载公开可用的模型。为了说明如何创建嵌入，我们将使用all-mpnet-base-v2模型。请注意，在第四章中，我们将进一步探讨如何为您的任务选择嵌入模型。

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector.shape

(768,)

这句话现在被编码在这个具有768个数值维度的单一向量中。在本书的第二部分，当我们开始探讨应用时，我们将开始看到这些文本嵌入向量在推动从分类到语义搜索再到检索增强生成（RAG）等一切方面的巨大作用。

# Word Embeddings Beyond LLMs


嵌入不仅在文本和语言生成领域有用，而且在许多其他领域也发挥着重要作用，包括推荐引擎和机器人技术。在这些领域中，嵌入（或为对象分配有意义的向量表示）同样非常有用。在本节中，我们将了解如何使用预训练的word2vec嵌入，并简要介绍该方法如何创建词嵌入。

了解word2vec的训练方式将为您学习第10章中的对比训练打下基础。然后在接下来的部分中，我们将看到这些嵌入如何用于推荐系统。

word2vec是一种流行的词嵌入技术，它能够将单词转换为密集的向量表示，这些向量能够捕捉单词之间的语义关系。通过训练，word2vec能够学习到单词之间的相似性，使得在向量空间中相似的单词具有相近的表示。

在推荐系统中，嵌入技术同样发挥着重要作用。通过将用户和物品转换为嵌入向量，推荐系统可以捕捉到用户和物品之间的潜在关系，从而为用户提供更加个性化的推荐。这种方法的优势在于能够处理大规模的数据集，并且能够捕捉到用户和物品之间的复杂关系。

总的来说，嵌入技术在多个领域中都发挥着重要作用，它们为处理复杂的数据关系提供了一种有效的方法。

In [ ]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


在这里，我们已经下载了基于维基百科训练的大量单词的嵌入。然后，我们可以通过查看某个特定单词（例如“king”，即“国王”）的最近邻来探索嵌入空间。

通过这种方法，我们可以观察到与“king”在语义上最为接近的其他单词。这些单词可能是与“king”具有相似含义、上下文或属性的词汇，如“queen”（女王）、“monarch”（君主）、“ruler”（统治者）等。这种探索嵌入空间的方式有助于我们理解嵌入模型如何捕捉单词之间的语义关系，并为我们提供了一种直观的方式来评估嵌入模型的质量。

需要注意的是，这里使用的嵌入模型是基于维基百科训练的，因此它可能更擅长处理与维基百科内容相关的单词和短语。在实际应用中，我们可能需要根据具体任务和数据集来选择合适的嵌入模型和训练方法。

In [ ]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

# The Word2vec Algorithm and Contrastive Training

论文《Efficient estimation of word representations in vector space》中详细描述了word2vec算法，而《The Illustrated Word2vec》则对其进行了生动的图解。在接下来讨论为推荐引擎创建嵌入的一种方法时，我们将在此基础上凝练其核心思想。

与大型语言模型（LLMs）一样，word2vec也是基于文本生成的示例进行训练的。例如，假设我们有弗兰克·赫伯特（Frank Herbert）的《沙丘》小说中的文本“Thou shalt not make a machine in the likeness of a human mind”（ “汝不可制造类似人类思维的机器”）。该算法使用滑动窗口来生成训练示例。例如，我们可以设置窗口大小为2，这意味着我们考虑中心词每侧的两个邻居。

嵌入是从分类任务中生成的。该任务用于训练神经网络，以预测单词是否通常出现在相同的上下文中（此处的上下文是指在我们建模的训练数据集中的许多句子中）。我们可以将其想象为一个神经网络，它接受两个单词作为输入，如果它们倾向于出现在相同的上下文中，则输出1，否则输出0。

在滑动窗口的第一个位置，我们可以生成四个训练示例，如图2-11所示。

![](8.png)

在您描述的训练例子中，每个例子中都会使用中心词作为一个输入，而它的每一个邻居词则作为另一个不同的输入。我们期望最终训练出的模型能够识别这种邻居关系，如果它接收到的两个输入词确实是邻居，则输出1。这些训练例子可以在图2-12中可视化表示。

![](9.png)

然而，如果我们只有一个目标值为1的数据集，那么模型可以通过始终输出1来作弊并取得好成绩。为了解决这个问题，我们需要用那些通常不是邻居的单词的例子来丰富我们的训练数据集。这些被称为负例，并展示在图2-13中。

![](10.png)

事实证明，我们在选择负例时不必过于讲究科学性。许多有用的模型都源自于从随机生成的例子中检测正例的简单能力（这一能力受到一个重要思想——噪声对比估计（Noise-Contrastive Estimation）的启发，并在“噪声对比估计：一种非规范化统计模型的新估计原理”中有所描述）。因此，在这种情况下，我们只需随机获取单词，并将它们添加到数据集中，同时标明它们不是邻居（因此，当模型看到这些单词对时，应该输出0）。
>为什么需要负采样在传统的词嵌入模型中，如Word2Vec，要计算每个词汇在上下文中的概率分布，需要使用softmax函数对整个词汇表进行运算。然而，当词汇表非常庞大时，全局计算softmax会变得非常耗时，因为其计算复杂度与词汇表的大小成正比。为了解决这个问题，负采样提出了一种更高效的训练策略。负采样的核心思想是将训练目标简化为仅关注正样本（即目标词汇的上下文词汇）和一小部分负样本（即从训练数据中未出现的词汇）之间的关系。相比于原始的softmax分类器需要更新整个权重矩阵，通过负采样这种方式，负采样大大减少了计算量，使得模型训练更加高效。

至此，我们已经了解了word2vec的两个主要概念（如图2-14所示）：跳字模型（skip-gram），即选择相邻单词的方法；以及负采样（negative sampling），即通过从数据集中随机采样来添加负例。

![](11.png)

我们可以从运行中的文本中生成数以百万计甚至数十亿计的此类训练示例。在继续在这个数据集上训练神经网络之前，我们需要做出一些分词决策，就像我们之前在处理大型语言模型（LLM）分词器时所看到的那样，这些决策包括如何处理大写字母和标点符号，以及我们希望在词汇表中有多少个分词。

然后，我们为每个分词创建一个嵌入向量，并随机初始化它们，如图2-15所示。在实际操作中，这是一个维度为vocab_size（词汇大小）x embedding_dimensions（嵌入维度）的矩阵。

这里，vocab_size指的是词汇表中分词的总数，而embedding_dimensions则是指每个嵌入向量的维度大小，即每个分词将被表示为一个具有固定长度的向量。

通过随机初始化这些嵌入向量，我们为神经网络的训练提供了一个起点。在训练过程中，网络将学习调整这些向量的值，以便更好地表示分词之间的语义关系和模式。这种嵌入表示方法使得神经网络能够处理和理解文本数据，从而在各种自然语言处理任务中表现出色。

![](12.png)

接下来，我们针对每个示例训练一个模型，该模型接收两个嵌入向量作为输入，并预测它们是否相关。我们可以在图2-16中看到这种模型的样子。

图2-16展示了一个神经网络，它被训练来预测两个单词是否是邻居。在训练过程中，它会更新嵌入向量，以产生最终的、经过训练的嵌入向量。根据模型的预测是否正确，典型的机器学习训练步骤会更新嵌入向量，以便下次模型接收到这两个向量时，有更大概率做出更正确的预测。在训练过程结束时，我们为词汇表中的所有分词获得了更好的嵌入表示。

这种接收两个向量并预测它们是否具有某种关系的模型思想是机器学习中最强大的思想之一，并且一次又一次地在语言模型中证明了其有效性。这就是为什么我们在第10章中专门讨论这个概念以及它如何针对特定任务（如句子嵌入和检索）优化语言模型的原因。同样的思想也是连接不同模态（如文本和图像）的核心，这是AI图像生成模型的关键，我们将在第9章关于多模态模型的讨论中看到这一点。在这种设置中，模型会接收到一张图像和一个标题，并应该预测该标题是否描述了该图像。

# Recommending songs by embeddings

正如我们之前提到的，嵌入的概念在许多其他领域都非常有用。在工业界，它被广泛应用于推荐系统中。

在推荐系统中，嵌入技术通常用于表示用户和物品（如电影、书籍、商品等）的特征。通过将这些用户和物品映射到高维空间中的向量，系统能够捕捉它们之间的潜在关系和相似性。这些嵌入向量随后被用于计算用户和物品之间的匹配程度，从而为用户推荐他们可能感兴趣的物品。

与语言模型中的嵌入类似，推荐系统中的嵌入也是通过学习得到的。系统会根据用户的历史行为、评分、点击等数据来训练嵌入向量，以便更好地反映用户和物品之间的实际关系。一旦训练完成，这些嵌入向量就可以被用于实时推荐、个性化推荐等任务中，从而提高推荐系统的准确性和用户满意度。

In [35]:
#training a song embedding model
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [36]:
len(playlists)#播放列表的个数

11088

In [ ]:
#播放列表
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [2]:
from gensim.models import Word2Vec
?Word2Vec

In [ ]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

sentences：可以是一个list，对于大语料集，建议使用BrownCorpus,Text8Corpus或lineSentence构建  
vector_size：是指词向量的维度，默认为100，依据词库的大小而实际改变  
window：滑窗大小，最开始我们本文举例的滑窗大小是2  
alpha: 初始学习速率  
min_count: 需要计算词向量的最小词频  
negative：即使用Negative Sampling时负采样的个数，默认是5。推荐在[3,10]之间  
sample: 高频词汇的随机降采样的配置阈值，默认为1e-3，范围是(0,1e-5)  
workers：用于控制训练的并行数  

In [ ]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9979680776596069),
 ('2640', 0.9964019060134888),
 ('3167', 0.9963980317115784),
 ('5549', 0.9959008693695068),
 ('2715', 0.9958351850509644),
 ('3117', 0.9954560995101929),
 ('2987', 0.9953479766845703),
 ('2881', 0.9951083660125732),
 ('2886', 0.9950577616691589),
 ('3094', 0.994985044002533)]

In [ ]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [ ]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2640,Red Barchetta,Rush
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses
2715,Rainbow In The Dark,Dio


In [ ]:
print_recommendations(2172)

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object
['2849' '2640' '3167' '5549' '2715']


,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2640,Red Barchetta,Rush
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses
2715,Rainbow In The Dark,Dio


In [ ]:
print_recommendations(842)

title     California Love (w\/ Dr. Dre & Roger Troutman)
artist                                              2Pac
Name: 842 , dtype: object
['5668' '413' '5661' '330' '886']


,title,artist
id,,
5668,How We Do (w\/ 50 Cent),The Game
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
5661,Sweet Dreams,Beyonce
330,Hate It Or Love It (w\/ 50 Cent),The Game
886,Heartless,Kanye West


“在本章中，我们介绍了大型语言模型（LLM）中的**标记（token）、分词器（tokenizer）以及使用标记嵌入（embedding）** 的有效方法。这为我们在下一章开始更深入地研究语言模型做好了准备，同时也为我们了解嵌入如何在语言模型之外得到应用打开了大门。”
“我们探讨了分词器是如何成为大型语言模型处理输入的第一步，将原始文本输入转换为标记ID。根据特定应用的具体要求，常见的分词方案包括将文本拆分为单词、子词标记、字符或字节。  
通过浏览现实世界中的预训练分词器（从BERT到GPT-2、GPT-4以及其他模型），我们了解了某些分词器在某些方面表现更好（例如，保留大写字母、换行符或其他语言的标记等信息）而其他方面只是彼此不同（例如，它们如何拆分某些单词）的领域。  
分词器设计的三个主要决策是分词算法（例如，BPE、WordPiece、SentencePiece）、分词参数（包括词汇量、特殊标记、大写字母的处理以及不同语言的处理）以及分词器训练所用的数据集。  
语言模型也是高质量上下文化标记嵌入的创造者，这些嵌入改进了原始静态嵌入。这些上下文化标记嵌入被用于命名实体识别（NER）、抽取式文本摘要和文本分类等任务。除了生成标记嵌入外，语言模型还可以生成覆盖整个句子甚至整个文档的文本嵌入。这为本书第二部分将介绍的语言模型应用中的大量应用提供了支持。  
在大型语言模型出现之前，word2vec、GloVe和fastText等词嵌入方法很受欢迎。在语言处理中，这些方法在很大程度上已被语言模型产生的上下文化词嵌入所取代。word2vec算法依赖于两个主要思想：跳字模型（skip-gram）和负采样（negative sampling）。它还使用了与我们在第10章中将看到的类似的对比训练。  
正如我们在从精选歌曲播放列表中构建的音乐推荐系统中讨论的那样，嵌入对于创建和改进推荐系统很有用。  
在下一章中，我们将深入探讨分词之后的过程：大型语言模型如何处理这些标记并生成文本？我们将研究一些使用Transformer架构的大型语言模型工作的主要直觉。”  